In [1]:
#библиотеки для парсинга
import requests as rq

from bs4 import BeautifulSoup

#библиотеки для предобработки
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize

In [8]:
import time
from tqdm import tqdm
from time import sleep

import numpy as np

In [9]:
#библиотеки для регрессии
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [4]:
# функция парсинг статьи и её заголовка
def GetArticle(url0):
    page0 = rq.get(url0)
    soup0 = BeautifulSoup(page0.text, features="html.parser")
    title = soup0.find_all('title')
    text_list = soup0.find_all('p')
    text = []
    for i in text_list:
        text.append(i.text)
    final_text = ' '.join(text)
    views = []
    views = soup0.find('div', {'class': 'widget-stat__item'}).text
    category = []
    category = soup0.find('div', {'class': 'page-main__rubric'}).text
    date = []
    date = soup0.find("meta", property="article:published_time")
    return url0, title, final_text, views, category, date

In [5]:
articles = []
z = 1
with tqdm(total=200) as pbar:
    while z <= 200:
        url = 'https://syuyumbike.ru/news?page=' + str(z)
        page = rq.get(url)
        soup = BeautifulSoup(page.text, features="html.parser")
        soup.find_all("a")
        titles = []
        for x in soup.find_all("a"): 
            titles.append(x.text)
        links = []
        for link in soup.find_all("a"):
            links.append(link.get('href')) 
        del links[:76]
        del links[15:]
        for link in links:
            new = GetArticle(link)
            articles.append(new)
        z += 1
        sleep(0.1)
        pbar.update(1)

100%|█████████████████████████████████████████| 200/200 [36:14<00:00, 10.87s/it]


In [5]:
url = 'https://syuyumbike.ru/news/otkrovenie/bu-ishekt-bkhet-yashmime'
page = rq.get(url)
soup = BeautifulSoup(page.text, features="html.parser")
likes = []
likes= soup.find_all('div', {'class': 'widget-stat__item'})
likes[3]

<div class="widget-stat__item">
<svg class="widget-stat__object" viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
<path d="M0 0h24v24H0z" fill="none"></path>
<path d="M12 4.5C7 4.5 2.73 7.61 1 12c1.73 4.39 6 7.5 11 7.5s9.27-3.11 11-7.5c-1.73-4.39-6-7.5-11-7.5zM12 17c-2.76 0-5-2.24-5-5s2.24-5 5-5 5 2.24 5 5-2.24 5-5 5zm0-8c-1.66 0-3 1.34-3 3s1.34 3 3 3 3-1.34 3-3-1.34-3-3-3z"></path>
</svg>
    4469
  </div>

In [6]:
likes[2]

<div class="widget-stat__item">
<svg class="widget-stat__object" viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
<path d="M0 0h24v24H0z" fill="none"></path>
<path d="M12 21.35l-1.45-1.32C5.4 15.36 2 12.28 2 8.5 2 5.42 4.42 3 7.5 3c1.74 0 3.41.81 4.5 2.09C13.09 3.81 14.76 3 16.5 3 19.58 3 22 5.42 22 8.5c0 3.78-3.4 6.86-8.55 11.54L12 21.35z"></path>
</svg>
              57
            </div>

In [6]:
art_df = pd.DataFrame(articles, columns = ['link', 'title', 'text', 'views', 'category', 'date'])
art_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   link      3000 non-null   object
 1   title     3000 non-null   object
 2   text      3000 non-null   object
 3   views     3000 non-null   object
 4   category  3000 non-null   object
 5   date      3000 non-null   object
dtypes: object(6)
memory usage: 140.8+ KB


In [15]:
#список всех категорий
classes = pd.value_counts(art_df["category"])
print(classes)

Күңелеңә җыйма            465
Яңалыклар                 432
Проза                     396
«Йолдызлы» яңалыклар      344
Белмәсәң бел              203
Бакча                     113
Тормыш кыйммәтләре         87
Актуаль тема               76
Арабыздан беребез          73
Дин                        68
Сәхнә һәм язмыш            68
Сәламәтлек                 63
Татар гаиләсе              47
Матур булыйк               46
«Көтә белгәннәр полкы»     42
Бала                       33
Name: category, dtype: int64


In [55]:
years = pd.value_counts(art_df['date'])
print(years)

2021    889
2020    758
2022    649
2023    260
Name: date, dtype: int64


In [10]:
art_df['date'].values[17]

'2023'

In [9]:
for index, row in art_df.iterrows():
    art_df['date'][index] = str(art_df['date'][index])
    art_df['date'][index] = art_df['date'][index].replace('<meta content="', '')
    art_df['date'][index] = art_df['date'][index].replace('" property="article:published_time"/>', '')
    a = art_df['date'][index]
    a = a[0:4]
    art_df['date'][index] = a

In [7]:
art_df['views'].values[17]

'\n\n\n\n\n\n              666\n            '

In [11]:
#очищаем от ненужных символов
for index, row in art_df.iterrows():
    art_df['text'][index] = art_df['text'][index].replace('\n', '')
    art_df['text'][index] = art_df['text'][index].replace('  ', '')
    art_df['views'][index] = art_df['views'][index].replace('\n', '')
    art_df['views'][index] = art_df['views'][index].replace('  ', '')
    art_df['category'][index] = art_df['category'][index].replace('\n', '')
    art_df['category'][index] = art_df['category'][index].replace('  ', '')
    art_df['text'][index] = art_df['text'][index].replace('\xa0', '')
    art_df['text'][index] = art_df['text'][index].replace('\n', '')
    art_df['text'][index] = art_df['text'][index].replace('\r', '')
    art_df['text'][index] = art_df['text'][index].replace('Татар хатын-кызлары өчен кызыклы язмаларны Сөембикә  Telegram-каналында  укыгыз', '')
    art_df['text'][index] = art_df['text'][index].replace('Фото', '')
    art_df['text'][index] = art_df['text'][index].replace('фото  ', '')
    art_df['text'][index] = art_df['text'][index].replace(': Анна Арахамия', '')
    art_df['text'][index] = art_df['text'][index].replace('фото: «Татар-информ» агентлыгы сайтыннан алынды.', '')
    art_df['text'][index] = art_df['text'][index].replace(':Изображение от lookstudio на Freepik', '')
    art_df['text'][index] = art_df['text'][index].replace('Татарстан Дәүләт Советы Рәисе Фәрит МӨХӘММӘТШИНның тормыш кагыйдәләре:http:///news/actual-tema/min-eshlgch-sz-tidererlek-bulmasyn', '')
    art_df['text'][index] = re.sub(r"[A-Za-z]+\.[A-Za-z]+", '', art_df['text'][index])
    
for index, row in art_df.iterrows():
    art_df['title'][index] = str(art_df['title'][index])
    art_df['title'][index] = art_df['title'][index].replace('[<title>', '')
    art_df['title'][index] = art_df['title'][index].replace('</title>]', '')

In [12]:
art_df[0:10]

,link,title,text,views,category,date
0,https://syuyumbike.ru/news/proza/kichekterelgn...,Кичектерелгән бәхет,Лена ХӘЙРУЛЛИНА Йокысы очты Гайсаның. Сәгатьтә...,45,Проза,2023
1,https://syuyumbike.ru/news/actual-tema/dlt-dum...,Дәүләт Думасында – «Туу тарихы»,Халыкара балаларны яклау көне уңаеннан Дәүләт ...,107,Актуаль тема,2023
2,https://syuyumbike.ru/news/yanalyklar/teatrda-...,Театрда – лекция-әңгәмә,Кәрим Тинчурин исемендәге Татар дәүләт драма һ...,97,Яңалыклар,2023
3,https://syuyumbike.ru/news/yanalyklar/vatanym-...,«Ватаным Татарстан» газетасы юбилее: «Без исән...,"«Россия матбугатының Алтын фонды»на кертелгән,...",139,Яңалыклар,2023
4,https://syuyumbike.ru/news/arabyzdan-berebez/k...,Китап акылны тәрбияли,«Китабыгыз тормышымны үзгәртте...» – дип еш яз...,191,Арабыздан беребез,2023
5,https://syuyumbike.ru/news/istiriya-zhurnala/a...,Абруй апа,Бу сәхифәдә дә – безнең журнал тарихы. Дөресрә...,164,Журнал тарихы,2023
6,https://syuyumbike.ru/news/belms-bel/khatyn-ky...,Хатын-кызны ниләр бәхетле итә?,«Үзең өчен бернәрсәне дә кызганырга – экономия...,290,Белмәсәң бел,2023
7,https://syuyumbike.ru/news/bakcha/kaen-degeten...,Каен дегетенең 10 файдасы,"Урамда – җәй, ә бакчада – корткычлар. Әле генә...",409,Бакча,2023
8,https://syuyumbike.ru/news/proza/sin-gen-kirk,Син генә кирәк,Азык-төлек тулы ике сумка күтәрепНадирә кайтып...,3542,Проза,2023
9,https://syuyumbike.ru/news/yanalyklar/berenche...,Беренче «КАРАШ» инсценировка мәктәбе,Габдулла Кариев исемендәге Казан татар дәүләт ...,202,Яңалыклар,2023


In [13]:
print(classes[:25])

\n\n        Күңелеңә җыйма\n      \n                   465
\n\n        Яңалыклар\n      \n                        432
\n\n        Проза\n      \n                            396
\n\n        «Йолдызлы» яңалыклар\n      \n             344
\n\n        Белмәсәң бел\n      \n                     203
\n\n        Бакча\n      \n                            113
\n\n        Тормыш кыйммәтләре\n      \n                87
\n\n        Актуаль тема\n      \n                      76
\n\n        Арабыздан беребез\n      \n                 73
\n\n        Дин\n      \n                               68
\n\n        Сәхнә һәм язмыш\n      \n                   68
\n\n        Сәламәтлек\n      \n                        63
\n\n        Татар гаиләсе\n      \n                     47
\n\n        Матур булыйк\n      \n                      46
\n\n        «Көтә белгәннәр полкы»\n      \n            42
\n\n        Татар гаиләсе / килен-кайнана\n      \n     38
\n\n        Бала\n      \n                              

In [14]:
freq_classes = ['Күңелеңә җыйма', 'Яңалыклар', 'Проза', '«Йолдызлы» яңалыклар', 'Белмәсәң бел', 'Бакча', 'Тормыш кыйммәтләре', 'Актуаль тема', 'Арабыздан беребез', 'Сәхнә һәм язмыш', 'Дин', 'Сәламәтлек', 'Татар гаиләсе', 'Матур булыйк', '«Көтә белгәннәр полкы»', 'Бала']
for index, row in art_df.iterrows():
    if art_df['category'][index] not in freq_classes:
        art_df.drop(labels = [index], axis = 0, inplace = True)
art_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2556 entries, 0 to 2999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   link      2556 non-null   object
 1   title     2556 non-null   object
 2   text      2556 non-null   object
 3   views     2556 non-null   object
 4   category  2556 non-null   object
 5   date      2556 non-null   object
dtypes: object(6)
memory usage: 204.3+ KB


In [10]:
#перемещаемся в папку библиотеки py_tat_morphan
import os
os.chdir('/Users/Zila/opt/anaconda3/lib/python3.9/site-packages/py_tat_morphan')
from py_tat_morphan.morphan import Morphan
morphan = Morphan()
print(morphan.lemma('урманнарга'))

['урман']


In [17]:
#список всех слов из словаря библиотеки
words = []
with open('py_tat_morphan/files/words.csv', 'r') as f:
    for i in f:
        t = i.split()
        words.append(t[0])

In [18]:
#cписок стоп-слов
tatar_stopwords = ['мин', 'син', 'без', 'сез', 'алар','кеше', 'кирәк', 'шуңа', 'күрә','ул', 'бару', 'кылу', 'һәм', 'белән', 'да', 'дә', 'бу', 'ул', 'дип', 'өчен', 'бер', 'иде', 'аның', 'буенча', 'ә', 'генә', 'бик', 'инде', 'шул', 'бар', 'гына', 'шулай', 'үз', 'ук', 'турында', 'алар', 'булган', 'булып', 'итеп', 'торган', 'әле', 'итә', 'дигән', 'аны', 'була', 'икән', 'бит', 'кебек', 'булды', 'кадәр', 'менә', 'ди', 'тагын', 'тә', 'килә', 'арасында', 'тора', 'килеп', 'тиеш', 'әмма', 'анда', 'диде', 'та', 'аларның', 'итү', 'килгән', 'һәр', 'үзе', 'шушы', 'исә', 'ала', 'мөмкин', 'ләкин', 'тик', 'аңа', 'шуңа', 'итте', 'бөтен', 'чөнки', 'нинди', 'ни', 'ничек', 'туры', 'бара', 'артык', 'төп']

stop_lemm = []
for i in range(len(tatar_stopwords)):
    word_lemm = morphan.lemma(tatar_stopwords[i])
    stop_lemm = stop_lemm + word_lemm
    
stop_lemm = list(set(stop_lemm))

print(stop_lemm)

['шул', 'да', 'бер', 'тә', 'әмма', 'шулай', 'тор', 'кыл', 'буенча', 'икән', 'үзе', 'чөнки', 'сез', 'һәм', 'белән', 'гына', 'генә', 'бик', 'ни', 'та', 'артык', 'алар', 'исә', 'и', 'бит', 'ти', 'бөтен', 'үз', 'бул', 'шушы', 'күрә', 'ләкин', 'кил', 'төп', 'бу', 'өчен', 'кеше', 'без', 'мөмкин', 'ул', 'дә', 'ничек', 'ала', 'кадәр', 'ара', 'кирәк', 'тур', 'нинди', 'арасында', 'тик', 'анда', 'әле', 'ис', 'ал', 'бар', 'ә', 'кебек', 'туры', 'ук', 'күр', 'мен', 'ди', 'һәр', 'инде', 'тагын', 'син', 'турында', 'ит', 'менә', 'мин', 'тиеш']


In [19]:
#лемматизация статей
art_df['lemmas'] = ''
for index, row in tqdm(art_df.iterrows()):
    text = art_df['text'][index]
    text = re.sub(r"^\s+|\s+$", "", text)
    
    text_low = text.lower()
        
    text_list_nltk = word_tokenize(text_low)
        
    text_without_punkt = [] 
    for word in text_list_nltk:
        if word[0].isalpha(): 
            text_without_punkt.append(word)
                
    tokens_lem = []
    tokens_except = []
    for i in text_without_punkt:
        if i in words:
            tokens_lem.append(morphan.lemma(i)[0])
        else:
            tokens_except.append(i)
                
    text_clean = []
    for word in tokens_lem:
        if word not in stop_lemm:
            text_clean.append(word)
    text_clean = ' '.join(text_clean)
    
    art_df['lemmas'][index] = text_clean
art_df[0:10]

2556it [2:10:44,  3.07s/it]


,link,title,text,views,category,date,lemmas
0,https://syuyumbike.ru/news/proza/kichekterelgn...,Кичектерелгән бәхет,Лена ХӘЙРУЛЛИНА Йокысы очты Гайсаның. Сәгатьтә...,45,Проза,2023,лена йокы оч сәгать кычкыр көт уян озак түшәм ...
1,https://syuyumbike.ru/news/actual-tema/dlt-dum...,Дәүләт Думасында – «Туу тарихы»,Халыкара балаларны яклау көне уңаеннан Дәүләт ...,107,Актуаль тема,2023,халыкара бала якла көн дәүләт дума чал фотогра...
2,https://syuyumbike.ru/news/yanalyklar/teatrda-...,Театрда – лекция-әңгәмә,Кәрим Тинчурин исемендәге Татар дәүләт драма һ...,97,Яңалыклар,2023,кәрим тинчурин исем татар дәүләт драма комедия...
3,https://syuyumbike.ru/news/yanalyklar/vatanym-...,«Ватаным Татарстан» газетасы юбилее: «Без исән...,"«Россия матбугатының Алтын фонды»на кертелгән,...",139,Яңалыклар,2023,россия матбугат алтын фонд на кер дәүләт иң зу...
4,https://syuyumbike.ru/news/arabyzdan-berebez/k...,Китап акылны тәрбияли,«Китабыгыз тормышымны үзгәртте...» – дип еш яз...,191,Арабыздан беребез,2023,китап тормыш үзгәр еш яз хәер китап тормыш баш...
6,https://syuyumbike.ru/news/belms-bel/khatyn-ky...,Хатын-кызны ниләр бәхетле итә?,«Үзең өчен бернәрсәне дә кызганырга – экономия...,290,Белмәсәң бел,2023,бернәрсә кызган экономия яра ишет хәзер хак кү...
7,https://syuyumbike.ru/news/bakcha/kaen-degeten...,Каен дегетенең 10 файдасы,"Урамда – җәй, ә бакчада – корткычлар. Әле генә...",409,Бакча,2023,урам җәй бакча корткыч яшелчә утыр тиш чык хәз...
8,https://syuyumbike.ru/news/proza/sin-gen-kirk,Син генә кирәк,Азык-төлек тулы ике сумка күтәрепНадирә кайтып...,3542,Проза,2023,азык-төлек тулы ике сумка кайт кер элгеч болга...
9,https://syuyumbike.ru/news/yanalyklar/berenche...,Беренче «КАРАШ» инсценировка мәктәбе,Габдулла Кариев исемендәге Казан татар дәүләт ...,202,Яңалыклар,2023,габдулла кариев исем казан татар дәүләт яшь та...
10,https://syuyumbike.ru/news/matur-bulyk/gardero...,Гардеробны баета торган җәйге биш кием,"Киемгә күпме генә акча сарыф итсәк тә, беркем...",496,Матур булыйк,2023,кием күп акча сарыф беркем арзан әлбәттә кыймм...


In [23]:
art_df.to_csv('articles.csv')

In [11]:
art_df = pd.read_csv('articles.csv')

In [12]:
art_df.index = np.arange(len(art_df))
art_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2556 entries, 0 to 2555
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2556 non-null   int64 
 1   link        2556 non-null   object
 2   title       2556 non-null   object
 3   text        2556 non-null   object
 4   views       2556 non-null   int64 
 5   category    2556 non-null   object
 6   date        2556 non-null   int64 
 7   lemmas      2556 non-null   object
dtypes: int64(3), object(5)
memory usage: 179.7+ KB


In [54]:
art_df.to_csv('articles_clean.csv')

In [89]:
art_df = pd.read_csv('articles_clean.csv')

In [23]:
art_df['likes'] = ''
for index, row in tqdm(art_df.iterrows()):
    url = art_df['link'][index]
    page = rq.get(url)
    soup = BeautifulSoup(page.text, features="html.parser")
    likes = []
    likes = soup.find_all('div', {'class': 'widget-stat__item'})
    art_df['likes'][index] = likes[2]

0it [00:00, ?it/s]/var/folders/ll/18sr221n7cs90nk8fdhgrwqw0000gq/T/ipykernel_1160/2958278832.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  art_df['likes'][index] = likes[2]
2556it [35:20,  1.21it/s]


In [22]:
art_df[0:10]

,Unnamed: 0.1,Unnamed: 0,link,title,text,views,category,date,lemmas
0,0,0,https://syuyumbike.ru/news/proza/kichekterelgn...,Кичектерелгән бәхет,Лена ХӘЙРУЛЛИНА Йокысы очты Гайсаның. Сәгатьтә...,45,Проза,2023,лена йокы оч сәгать кычкыр көт уян озак түшәм ...
1,1,1,https://syuyumbike.ru/news/actual-tema/dlt-dum...,Дәүләт Думасында – «Туу тарихы»,Халыкара балаларны яклау көне уңаеннан Дәүләт ...,107,Актуаль тема,2023,халыкара бала якла көн дәүләт дума чал фотогра...
2,2,2,https://syuyumbike.ru/news/yanalyklar/teatrda-...,Театрда – лекция-әңгәмә,Кәрим Тинчурин исемендәге Татар дәүләт драма һ...,97,Яңалыклар,2023,кәрим тинчурин исем татар дәүләт драма комедия...
3,3,3,https://syuyumbike.ru/news/yanalyklar/vatanym-...,«Ватаным Татарстан» газетасы юбилее: «Без исән...,"«Россия матбугатының Алтын фонды»на кертелгән,...",139,Яңалыклар,2023,россия матбугат алтын фонд на кер дәүләт иң зу...
4,4,4,https://syuyumbike.ru/news/arabyzdan-berebez/k...,Китап акылны тәрбияли,«Китабыгыз тормышымны үзгәртте...» – дип еш яз...,191,Арабыздан беребез,2023,китап тормыш үзгәр еш яз хәер китап тормыш баш...
5,5,6,https://syuyumbike.ru/news/belms-bel/khatyn-ky...,Хатын-кызны ниләр бәхетле итә?,«Үзең өчен бернәрсәне дә кызганырга – экономия...,290,Белмәсәң бел,2023,бернәрсә кызган экономия яра ишет хәзер хак кү...
6,6,7,https://syuyumbike.ru/news/bakcha/kaen-degeten...,Каен дегетенең 10 файдасы,"Урамда – җәй, ә бакчада – корткычлар. Әле генә...",409,Бакча,2023,урам җәй бакча корткыч яшелчә утыр тиш чык хәз...
7,7,8,https://syuyumbike.ru/news/proza/sin-gen-kirk,Син генә кирәк,Азык-төлек тулы ике сумка күтәрепНадирә кайтып...,3542,Проза,2023,азык-төлек тулы ике сумка кайт кер элгеч болга...
8,8,9,https://syuyumbike.ru/news/yanalyklar/berenche...,Беренче «КАРАШ» инсценировка мәктәбе,Габдулла Кариев исемендәге Казан татар дәүләт ...,202,Яңалыклар,2023,габдулла кариев исем казан татар дәүләт яшь та...
9,9,10,https://syuyumbike.ru/news/matur-bulyk/gardero...,Гардеробны баета торган җәйге биш кием,"Киемгә күпме генә акча сарыф итсәк тә, беркем...",496,Матур булыйк,2023,кием күп акча сарыф беркем арзан әлбәттә кыймм...


In [53]:
art_df['likes'][2000]

5

In [43]:
for index, row in art_df.iterrows():
    art_df['likes'][index] = str(art_df['likes'][index])
    art_df['likes'][index] = art_df['likes'][index].replace('\n', '')
    art_df['likes'][index] = art_df['likes'][index].replace('<div class="widget-stat__item">', '')
    art_df['likes'][index] = art_df['likes'][index].replace('<svg', '')
    art_df['likes'][index] = art_df['likes'][index].replace('viewBox="0 0 24 24"', '')
    art_df['likes'][index] = art_df['likes'][index].replace('viewbox="0 0 24 24"', '')
    art_df['likes'][index] = art_df['likes'][index].replace('xmlns="http://www.w3.org/2000/svg"', '')
    art_df['likes'][index] = art_df['likes'][index].replace('class="widget-stat__object"', '')
    art_df['likes'][index] = art_df['likes'][index].replace('>', '')
    art_df['likes'][index] = art_df['likes'][index].replace('<path d="M0 0h24v24H0z"', '')
    art_df['likes'][index] = art_df['likes'][index].replace('fill="none"', '')
    art_df['likes'][index] = art_df['likes'][index].replace('</path', '')
    art_df['likes'][index] = art_df['likes'][index].replace('<path', '')
    art_df['likes'][index] = art_df['likes'][index].replace('d="M12 21.35l-1.45-1.32C5.4 15.36 2 12.28 2 8.5 2 5.42 4.42 3 7.5 3c1.74 0 3.41.81 4.5 2.09C13.09 3.81 14.76 3 16.5 3 19.58 3 22 5.42 22 8.5c0 3.78-3.4 6.86-8.55 11.54L12 21.35z"', '')
    art_df['likes'][index] = art_df['likes'][index].replace('</svg', '')
    art_df['likes'][index] = art_df['likes'][index].replace('</div>', '')
    art_df['likes'][index] = art_df['likes'][index].replace('</div', '')
    art_df['likes'][index] = art_df['likes'][index].replace(' ', '')

/var/folders/ll/18sr221n7cs90nk8fdhgrwqw0000gq/T/ipykernel_1160/4288118578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  art_df['likes'][index] = str(art_df['likes'][index])
/var/folders/ll/18sr221n7cs90nk8fdhgrwqw0000gq/T/ipykernel_1160/4288118578.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  art_df['likes'][index] = art_df['likes'][index].replace('\n', '')
/var/folders/ll/18sr221n7cs90nk8fdhgrwqw0000gq/T/ipykernel_1160/4288118578.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [52]:
for index, row in art_df.iterrows():
    art_df['likes'][index] = int(art_df['likes'][index])

/var/folders/ll/18sr221n7cs90nk8fdhgrwqw0000gq/T/ipykernel_1160/3006981102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  art_df['likes'][index] = int(art_df['likes'][index])


In [ ]:
LINEAR REGRESSION

In [90]:
art_df.isna().any()

Unnamed: 0.2    False
Unnamed: 0.1    False
Unnamed: 0      False
link            False
title           False
text            False
views           False
category        False
date            False
lemmas          False
likes           False
dtype: bool

In [91]:
for index, row in art_df.iterrows():
    a = art_df['lemmas'][index]
    a = a.split()
    art_df['lemmas'][index] = a

/var/folders/ll/18sr221n7cs90nk8fdhgrwqw0000gq/T/ipykernel_1160/1682762391.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  art_df['lemmas'][index] = a


In [92]:
# создаем лист листов(описаний)
input_text = list(art_df.lemmas.values) 
input_text[1]

['халыкара',
 'бала',
 'якла',
 'көн',
 'дәүләт',
 'дума',
 'чал',
 'фотограф',
 'күргәзмә',
 'ач',
 'ту',
 'ата',
 'уникаль',
 'ту',
 'көн',
 'туй',
 'чыгар',
 'кичә',
 'фото',
 'төшер',
 'тормыш',
 'иң',
 'көн',
 'ту',
 'соң',
 'әни',
 'бала',
 'ту',
 'көт',
 'яңа',
 'ту',
 'сак',
 'кабул',
 'үлчә',
 'биләүлә',
 'әти',
 'тапкыр',
 'бала',
 'кул',
 'әлеге',
 'мизгел',
 'мәңгелек',
 'кал',
 'проект',
 'автор',
 'фотограф',
 'берничә',
 'ел',
 'элек',
 'чал',
 'кама',
 'бала',
 'медицина',
 'үзәк',
 'бала',
 'ту',
 'мизгел',
 'фото',
 'төшер',
 'башла',
 'проект',
 'автор',
 'ту',
 'мәл',
 'фото',
 'төшер',
 'бала',
 'быел',
 'яшь',
 'тул',
 'хан',
 'яшә',
 'ике',
 'бала',
 'әни',
 'экспозиция',
 'иң',
 'як',
 'эмоциональ',
 'фото',
 'тәкъдим',
 'проект',
 'өс',
 'эшлә',
 'меңләгән',
 'кадр',
 'төшер',
 'яңа',
 'ту',
 'чик',
 'ата-ана',
 'бәхет',
 'ике',
 'дубль',
 'яки',
 'эчкерсез',
 'түгел',
 'шок',
 'гаҗәплән',
 'бәхет',
 'күргәзмә',
 'ое',
 'тәкъдим',
 'соң',
 'шундый',
 'тәртип',


In [93]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]

documents[10]

TaggedDocument(words=['сора', 'республика', 'клиник', 'хастаханә', 'лилия', 'җавап', 'бир', 'еш', 'кына', 'гөмбә', 'борчы', 'табиб', 'түгел', 'ир', 'дәвала', 'түгел', 'ми', 'берни', 'борчы', 'нәрсә', 'китер', 'йә', 'гөмбә', 'китер', 'чыгар', 'авыр', 'ир', 'хатын-кыз', 'гормональ', 'фон', 'анатомик', 'төз', 'үзенчәлек', 'сәбәп', 'еш', 'очра', 'җенси', 'юл', 'күч', 'ир', 'дәвала', 'зарур', 'симптом', 'диярлек', 'иммунитет', 'какша', 'йә', 'хроник', 'авыр', 'тышкы', 'җенси', 'ор', 'ялкынсын', 'кыз', 'кычыт', 'аш', 'сиз', 'баш', 'дәвала', 'бәвел', 'бүл', 'чыгар', 'зыян', 'сизгер', 'ки', 'чират', 'китер', 'авыр', 'аз', 'аерым', 'очрак', 'бөер', 'баш', 'йөрәк', 'эчке', 'китер', 'хатын-кыз', 'кайсы', 'белгеч', 'мөрәҗәгать', 'симптом', 'күпчелек', 'хатын-кыз', 'гинеколог', 'мөрәҗәгать', 'бәвел', 'уролог', 'дәвала', 'хатын-кыз', 'ир', 'соң', 'хатын-кыз', 'үзенчәлек', 'тот', 'уролог', 'гинеколог', 'җибәр', 'аннары', 'кайбер', 'очрак', 'баш', 'ун', 'көн', 'соңгы', 'биш', 'көн', 'бераз', 'кал', 'б

In [94]:
Векторизация текстов

SyntaxError: invalid syntax (3166493384.py, line 1)

In [95]:
model_D2V = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [96]:
model_D2V.dv[documents[999].tags[0]]

array([ 0.96467966,  0.8353062 ,  2.3239172 , -0.77247685,  2.6799526 ],
      dtype=float32)

In [ ]:
Векторизация названий категорий

In [97]:
input_cat = list(art_df.category.values) 
input_cat[1]

'Актуаль тема'

In [98]:
categories = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_cat)]

# посмотрим на пример разметки
categories[10]

TaggedDocument(words='Сәламәтлек', tags=[10])

In [99]:
model_D2V_cat = Doc2Vec(categories, vector_size=1, window=2, min_count=1, workers=4)
model_D2V_cat.dv[documents[999].tags[0]]

array([-0.42536518], dtype=float32)

In [100]:
# создадим список с векторами для каждого текста
vectors = []
for x in documents:
    vec = list(model_D2V.dv[x.tags][0])
    vectors.append(vec)

In [101]:
# датафрейм, где все компоненты векторов в отдельных столбцах
split_df = pd.DataFrame(vectors,
                        columns=['v1', 'v2', 'v3','v4',"v5"])

split_df

,v1,v2,v3,v4,v5
0,0.022576,1.563948,-0.986830,2.600627,3.478758
1,1.008133,-1.529106,0.978324,-1.824237,0.600243
2,1.422564,-0.517376,0.620144,-0.514320,0.304747
3,2.081814,-1.713754,2.007909,-1.573798,0.338485
4,1.165151,-0.982454,1.824276,-0.652952,0.859366
...,...,...,...,...,...
2551,-0.145244,-0.381283,1.048570,0.921151,1.936211
2552,-0.509209,1.318038,-0.295001,1.839768,3.170854
2553,0.704126,-2.708469,-0.111667,-0.229975,0.441494
2554,-0.142995,-3.384969,-1.658065,-3.236372,1.802213


In [102]:
# добавление его к основному датафрейму
art_df= art_df.join(split_df, how='left')
print(art_df.shape)
art_df.head()

(2556, 16)


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,link,title,text,views,category,date,lemmas,likes,v1,v2,v3,v4,v5
0,0,0,0,https://syuyumbike.ru/news/proza/kichekterelgn...,Кичектерелгән бәхет,Лена ХӘЙРУЛЛИНА Йокысы очты Гайсаның. Сәгатьтә...,45,Проза,2023,"[лена, йокы, оч, сәгать, кычкыр, көт, уян, оза...",8,0.022576,1.563948,-0.986830,2.600627,3.478758
1,1,1,1,https://syuyumbike.ru/news/actual-tema/dlt-dum...,Дәүләт Думасында – «Туу тарихы»,Халыкара балаларны яклау көне уңаеннан Дәүләт ...,107,Актуаль тема,2023,"[халыкара, бала, якла, көн, дәүләт, дума, чал,...",1,1.008133,-1.529106,0.978324,-1.824237,0.600243
2,2,2,2,https://syuyumbike.ru/news/yanalyklar/teatrda-...,Театрда – лекция-әңгәмә,Кәрим Тинчурин исемендәге Татар дәүләт драма һ...,97,Яңалыклар,2023,"[кәрим, тинчурин, исем, татар, дәүләт, драма, ...",0,1.422564,-0.517376,0.620144,-0.514320,0.304747
3,3,3,3,https://syuyumbike.ru/news/yanalyklar/vatanym-...,«Ватаным Татарстан» газетасы юбилее: «Без исән...,"«Россия матбугатының Алтын фонды»на кертелгән,...",139,Яңалыклар,2023,"[россия, матбугат, алтын, фонд, на, кер, дәүлә...",0,2.081814,-1.713754,2.007909,-1.573798,0.338485
4,4,4,4,https://syuyumbike.ru/news/arabyzdan-berebez/k...,Китап акылны тәрбияли,«Китабыгыз тормышымны үзгәртте...» – дип еш яз...,191,Арабыздан беребез,2023,"[китап, тормыш, үзгәр, еш, яз, хәер, китап, то...",1,1.165151,-0.982454,1.824276,-0.652952,0.859366


In [103]:
# список векторов категорий
vectors_cat = []
for x in categories:
    vec_cat = list(model_D2V_cat.dv[x.tags][0])
    vectors_cat.append(vec_cat)

In [104]:
# датафрейм с векторами категорий
split_df_cat = pd.DataFrame(vectors_cat,
                        columns=['v1_cat'])

split_df_cat

,v1_cat
0,-0.499023
1,-0.583615
2,-0.943059
3,0.757831
4,0.273275
...,...
2551,0.736593
2552,0.122016
2553,-0.794148
2554,0.142139


In [105]:
# теперь добавим его к основному датафрейму
art_df= art_df.join(split_df_cat, how='left')
print(art_df.shape)
art_df.head()

(2556, 17)


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,link,title,text,views,category,date,lemmas,likes,v1,v2,v3,v4,v5,v1_cat
0,0,0,0,https://syuyumbike.ru/news/proza/kichekterelgn...,Кичектерелгән бәхет,Лена ХӘЙРУЛЛИНА Йокысы очты Гайсаның. Сәгатьтә...,45,Проза,2023,"[лена, йокы, оч, сәгать, кычкыр, көт, уян, оза...",8,0.022576,1.563948,-0.986830,2.600627,3.478758,-0.499023
1,1,1,1,https://syuyumbike.ru/news/actual-tema/dlt-dum...,Дәүләт Думасында – «Туу тарихы»,Халыкара балаларны яклау көне уңаеннан Дәүләт ...,107,Актуаль тема,2023,"[халыкара, бала, якла, көн, дәүләт, дума, чал,...",1,1.008133,-1.529106,0.978324,-1.824237,0.600243,-0.583615
2,2,2,2,https://syuyumbike.ru/news/yanalyklar/teatrda-...,Театрда – лекция-әңгәмә,Кәрим Тинчурин исемендәге Татар дәүләт драма һ...,97,Яңалыклар,2023,"[кәрим, тинчурин, исем, татар, дәүләт, драма, ...",0,1.422564,-0.517376,0.620144,-0.514320,0.304747,-0.943059
3,3,3,3,https://syuyumbike.ru/news/yanalyklar/vatanym-...,«Ватаным Татарстан» газетасы юбилее: «Без исән...,"«Россия матбугатының Алтын фонды»на кертелгән,...",139,Яңалыклар,2023,"[россия, матбугат, алтын, фонд, на, кер, дәүлә...",0,2.081814,-1.713754,2.007909,-1.573798,0.338485,0.757831
4,4,4,4,https://syuyumbike.ru/news/arabyzdan-berebez/k...,Китап акылны тәрбияли,«Китабыгыз тормышымны үзгәртте...» – дип еш яз...,191,Арабыздан беребез,2023,"[китап, тормыш, үзгәр, еш, яз, хәер, китап, то...",1,1.165151,-0.982454,1.824276,-0.652952,0.859366,0.273275


In [106]:
# переопределение датасета

features = art_df[['views', 'date', 'likes', "v1","v2","v3","v4","v5", "v1_cat"]]

features.head(5)

,views,date,likes,v1,v2,v3,v4,v5,v1_cat
0,45,2023,8,0.022576,1.563948,-0.986830,2.600627,3.478758,-0.499023
1,107,2023,1,1.008133,-1.529106,0.978324,-1.824237,0.600243,-0.583615
2,97,2023,0,1.422564,-0.517376,0.620144,-0.514320,0.304747,-0.943059
3,139,2023,0,2.081814,-1.713754,2.007909,-1.573798,0.338485,0.757831
4,191,2023,1,1.165151,-0.982454,1.824276,-0.652952,0.859366,0.273275


In [27]:
РАЗДЕЛЕНИЕ ДЛЯ ОБУЧЕНИЯ И ДЛЯ ТЕСТА

SyntaxError: invalid syntax (756632828.py, line 1)

In [107]:
# определение X и y

X = features.drop(["likes"],axis=1) 

X.shape
X.head()

y = art_df["likes"]

In [108]:
# разделение датасета на 4 части 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

# посмотрим на размерности
print(f"X_train shape:{X_train.shape},\
      X_test shape:{X_test.shape}")

print(f"y_train shape:{y_train.shape},\
      y_test shape:{y_test.shape}")

X_train shape:(1917, 8),      X_test shape:(639, 8)
y_train shape:(1917,),      y_test shape:(639,)


In [109]:
ОБУЧЕНИЕ АЛГОРИТМА РЕГРЕССИИ

SyntaxError: invalid syntax (1590886550.py, line 1)

In [116]:
regressor_1 = Ridge(alpha=20.0) 

regressor_1.fit(X_train, y_train)

Ridge(alpha=20.0)

In [117]:
y_preds = regressor_1.predict(X_test)

In [118]:
mean_absolute_error(y_test, y_preds) 

7.7258539286240975